In [1]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

In [5]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='X592PAoN2yPC_PGQbDW564k6LKamTK39rv9YdtQpofQW',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'customclouddeploymentliverdisease-donotdelete-pr-ehggnijce6fazf'
object_key = 'indian_liver_patient.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset = pd.read_csv(body)
dataset.head()


,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [6]:

import pandas as pd
import numpy as np


In [7]:
dataset['Albumin_and_Globulin_Ratio'] = dataset['Albumin_and_Globulin_Ratio'].fillna(dataset['Albumin_and_Globulin_Ratio'].median())


In [8]:
dataset['Gender'] = np.where(dataset['Gender']=='Male', 1,0)

In [9]:
dataset = dataset.drop('Direct_Bilirubin', axis=1)


In [10]:

X=dataset.drop(['Dataset'],axis='columns')
y=dataset['Dataset']


In [11]:
X.head()

,Age,Gender,Total_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio
0,65,0,0.7,187,16,18,6.8,3.3,0.90
1,62,1,10.9,699,64,100,7.5,3.2,0.74
2,62,1,7.3,490,60,68,7.0,3.3,0.89
3,58,1,1.0,182,14,20,6.8,3.4,1.00
4,72,1,3.9,195,27,59,7.3,2.4,0.40


In [12]:
#y = dataset.iloc[:, -1]
y.head()


0    1
1    1
2    1
3    1
4    1
Name: Dataset, dtype: int64

In [13]:
dataset.isnull().sum()

Age                           0
Gender                        0
Total_Bilirubin               0
Alkaline_Phosphotase          0
Alamine_Aminotransferase      0
Aspartate_Aminotransferase    0
Total_Protiens                0
Albumin                       0
Albumin_and_Globulin_Ratio    0
Dataset                       0
dtype: int64

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=50)



In [15]:

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=40)
model.fit(x_train, y_train)


RandomForestClassifier(random_state=40)

In [16]:

from sklearn.metrics import accuracy_score
y_predict = model.predict(x_test)
accuracy_score(y_test, y_predict)


0.6923076923076923

In [17]:

pd.crosstab(y_test, y_predict)


col_0,1,2
Dataset,,
1,70,6
2,30,11


In [18]:

get_ipython().system('pip install ibm_watson_machine_learning')


In [19]:
from ibm_watson_machine_learning import APIClient
import json


In [20]:
wml_credentials = {
    "apikey":"pHpCTQV4eHfoHFEFKFZ-BnFiH36qSuXPk8xkTiUGOMpm",
    "url":"https://us-south.ml.cloud.ibm.com"
}

In [21]:
wml_client = APIClient(wml_credentials)
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ------  ------------------------
ID                                    NAME    CREATED
155e1e50-59d4-4e6e-a852-dffca3b1badd  models  2022-11-13T15:11:37.091Z
------------------------------------  ------  ------------------------


In [23]:

Space_Id="155e1e50-59d4-4e6e-a852-dffca3b1badd"


In [24]:

wml_client.set.default_space(Space_Id)


'SUCCESS'

In [25]:

wml_client.software_specifications.list(500)


-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [26]:

import sklearn
sklearn.__version__


'1.0.2'

In [27]:
pip install scikit-learn==0.24


     |████████████████████████████████| 23.8 MB 16.7 MB/s eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autoai-ts-libs 1.1.9 requires scikit-learn<1.1,>=1.0.2; python_version >= "3.9", but you have scikit-learn 0.24.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [28]:
MODEL_NAME = 'liver_disease'
DEPLOYMENT_NAME = 'liver_disease'
DEMO_MODEL = model


In [29]:

software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [30]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}


In [ ]:
import joblib
model_details = wml_client.repository.store_model(
    model=DEMO_MODEL, 
    meta_props=model_props, 
    training_data=x_train, 
    training_target=y_train
)

In [32]:


model_details


{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'Dataset',
  'schemas': {'input': [{'fields': [{'name': 'Age', 'type': 'int64'},
      {'name': 'Gender', 'type': 'int64'},
      {'name': 'Total_Bilirubin', 'type': 'float64'},
      {'name': 'Alkaline_Phosphotase', 'type': 'int64'},
      {'name': 'Alamine_Aminotransferase', 'type': 'int64'},
      {'name': 'Aspartate_Aminotransferase', 'type': 'int64'},
      {'name': 'Total_Protiens', 'type': 'float64'},
      {'name': 'Albumin', 'type': 'float64'},
      {'name': 'Albumin_and_Globulin_Ratio', 'type': 'float64'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'scikit-learn_1.0'},
 'metadata': {'created_at': '2022-11-17T14:09:03.999Z',
  'id': 'a199ca22-da1e-4d8c-8541-60e641b3b66d',
  'modified_at': '2022-11-17T14:09:07.153Z',
  'name': 'liver_disease',
  'owner': 'IBMid-662003V6GP',
  'resource

In [33]:


model_id = wml_client.repository.get_model_id(model_details)
model_id


'a199ca22-da1e-4d8c-8541-60e641b3b66d'

In [34]:
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [35]:
deployment = wml_client.deployments.create(
    artifact_uid=model_id, 
    meta_props=deployment_props 
)





#######################################################################################

Synchronous deployment creation for uid: 'a199ca22-da1e-4d8c-8541-60e641b3b66d' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='a39085b5-e117-46c1-805d-4e3df71d7047'
------------------------------------------------------------------------------------------------


